"""t0-H,L
	$174.71	$173.45
"""

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from copy import copy

In [2]:
path1 = "./data/1/"
path2 = "./data/2/"
path3 = "./data/3/"
path4 = "./data/4/"
path5 = "./data/5/"
path6 = "./data/UpToDate/"

In [3]:
KP = 80
KF = 25

In [4]:
def fix_dollars(apple2):
    new_open = []
    new_close = []
    new_high = []
    new_low = []
    for i in range(len(apple2['Open'])):
        new_open.append(float(apple2['Open'][i][1:]))
        new_close.append(float(apple2['Close/Last'][i][1:]))
        new_high.append(float(apple2['High'][i][1:]))
        new_low.append(float(apple2['Low'][i][1:]))
    

    apple2.drop('Open',axis=1,inplace=True)
    apple2.drop('Close/Last',axis=1,inplace=True)
    apple2.drop('High',axis=1,inplace=True)
    apple2.drop('Low',axis=1,inplace=True)
    

    apple2['Close'] = new_close
    apple2['Open'] = new_open
    apple2['High'] = new_high
    apple2['Low'] = new_low
    
  

In [5]:
def fix_date(apple2):
    month = []
    year = []
    week_start = []
    week_end = []
    flag = False
    for i in range(len(apple2['Date'])):
        month.append(int(apple2['Date'][i][0:2]))
        year.append(int(apple2['Date'][i][6:]))
        today = int(apple2['Date'][i][3:5])
        
        if i == 0:
            tomorrow = int(apple2['Date'][i + 1][3:5])
            if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i + 1][3:5]) - 1 == int(apple2['Date'][i + 2][3:5]) - 2 \
                    == int(apple2['Date'][i + 3][3:5]) - 3 == int(apple2['Date'][i + 4][3:5]) - 4:
                week_start.append(True)
                week_end.append(False)
            elif today + 1 == tomorrow:
                week_start.append(False)
                week_end.append(False)
            else:
                week_start.append(False)
                week_end.append(True)
                flag = True
                continue
        else:
            try:
                tomorrow = int(apple2['Date'][i + 1][3:5])
                if today + 1 == tomorrow:  # I should really be cross referencing with a calendar api, not hacking this together
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(False)
                    week_end.append(True)
                    flag = True
                    continue
                if flag:
                    week_start[-1] = True
                    flag = False
            except:
                yesterday = int(apple2['Date'][i - 1][3:5])
                if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i - 1][3:5]) + 1 == int(
                        apple2['Date'][i - 2][3:5]) + 2 \
                        == int(apple2['Date'][i - 3][3:5]) + 3 == int(apple2['Date'][i - 4][3:5]) + 4:
                    week_start.append(False)
                    week_end.append(True)
                elif yesterday + 1 == today:
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(True)
                    week_end.append(False)

    apple2.drop('Date', axis=1, inplace=True)

    apple2['Month'] = month
    apple2['Year'] = year
    apple2['WeekStart'] = week_start
    apple2['WeekEnd'] = week_end


In [6]:
def get_targets(df,KF):
    target_percent_inc = []
    target_best_day = []
    target_worst_sell = []
    target_day_high = []
    target_day_low = []
    options = ['t--'+str(i)+'Low' for i in range(1,KF+1)]
    for i in range(len(df['t-0Low'])):
        today = df['t-0High'][i]
        target_day_high.append(today)
        today_worst = df['t-0Low'][i]
        target_day_low.append(today_worst)
        best = np.max([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])
        worst = np.max([df['t--'+str(j)+'High'][i] for j in range(1,KF+1)])
        best_day = np.argmax([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])+1
        target_percent_inc.append(round((best-today)*100/today,0))
        target_worst_sell.append(round((worst-today_worst)*100/today,0))
        target_best_day.append(best_day)
    return target_percent_inc, target_best_day, target_worst_sell,target_day_high,target_day_low

In [7]:
def fix_shift(apple2,KP,KF,updated=False):
    #KP = 50
    #changing KP and KF from 10 to 30 causes accuracy to drop...Why? Possibly because there are now more classes for y
    #KF = 25

    names = ['Volume','Close','Open','High','Low','Month','Year'] 
    cases = pd.DataFrame(apple2.drop('WeekStart',axis=1).drop('WeekEnd',axis=1).values)
    df = pd.concat([cases.shift(i) for i in range(KP-int(updated),-KF-1,-1)],axis=1).join(apple2['WeekStart']).join(apple2['WeekEnd'])
    df.columns = ['t-'+str(i+int(updated))+x for i in range(KP-int(updated),-KF-1,-1) for x in names]+['WeekStart','WeekEnd']
    return df

In [8]:
def fix_normalize(apple2,df):
    for apple_col in list(apple2):
        if apple_col in ['WeekStart','WeekEnd']:
            continue
        MEAN = apple2[apple_col].mean()
        STD = apple2[apple_col].std()
        for df_col in list(df):
            if apple_col in df_col:
                if apple_col =='Month' or apple_col == 'Year':
                    df = df.astype({df_col: 'int32'})
                else:
                    df[df_col]=round((df[df_col]-MEAN)/STD,0)

In [9]:
def drop_future(df,KF):
    for i in range(0,-KF-1,-1):
        for x in ['Month','Year','Volume','Close','Open','High','Low']:
            df.drop('t-'+str(i)+x,axis=1,inplace=True)

In [10]:
def fix_reverse(apple2):
    return apple2.loc[::-1].reset_index(drop=True)

In [11]:
apple = pd.read_csv(path5+"aapl_historical.csv")
apple = apple.iloc[:450]
apple2 = apple.copy(deep=True)
fix_dollars(apple2)
apple2 = fix_reverse(apple2)
apple2

,Date,Volume,Close,Open,High,Low
0,08/02/2021,62879960,145.520,146.36,146.950,145.25
1,08/03/2021,64786620,147.360,145.81,148.045,145.18
2,08/04/2021,56368270,146.950,147.27,147.790,146.28
3,08/05/2021,46397670,147.060,146.98,147.840,146.17
4,08/06/2021,54126810,146.140,146.35,147.110,145.63
...,...,...,...,...,...,...
445,05/09/2023,45326870,171.770,173.05,173.540,171.60
446,05/10/2023,53724500,173.555,173.02,174.030,171.90
447,05/11/2023,49514680,173.750,173.85,174.590,172.17
448,05/12/2023,45533140,172.570,173.62,174.060,171.00


In [12]:
fix_date(apple2)
apple2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,62879960,145.520,146.36,146.950,145.25,8,2021,True,False
1,64786620,147.360,145.81,148.045,145.18,8,2021,False,False
2,56368270,146.950,147.27,147.790,146.28,8,2021,False,False
3,46397670,147.060,146.98,147.840,146.17,8,2021,False,False
4,54126810,146.140,146.35,147.110,145.63,8,2021,False,True
...,...,...,...,...,...,...,...,...,...
445,45326870,171.770,173.05,173.540,171.60,5,2023,False,False
446,53724500,173.555,173.02,174.030,171.90,5,2023,False,False
447,49514680,173.750,173.85,174.590,172.17,5,2023,False,False
448,45533140,172.570,173.62,174.060,171.00,5,2023,False,True


In [13]:
df = fix_shift(apple2,KP,KF)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021.0,82278260.0,156.69,154.97,157.26,154.390,9.0,2021.0,True,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021.0,74420210.0,155.11,156.98,157.04,153.975,9.0,2021.0,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021.0,57305730.0,154.07,155.49,156.11,153.950,9.0,2021.0,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021.0,140893200.0,148.97,155.00,155.48,148.700,9.0,2021.0,False,False
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021.0,102404300.0,149.55,150.63,151.42,148.750,9.0,2021.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,71379650.0,133.41,133.880,134.26,131.44,1.0,2023.0,57809720.0,134.76,132.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
446,57809720.0,134.76,132.030,134.92,131.66,1.0,2023.0,63646630.0,135.94,134.830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
447,63646630.0,135.94,134.830,137.29,134.13,1.0,2023.0,69672800.0,135.21,136.815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
448,69672800.0,135.21,136.815,138.61,135.03,1.0,2023.0,58280410.0,135.27,134.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True


In [14]:
def fix_null(df):
    df.dropna(inplace=True)
    return df.reset_index(drop=True)
    

In [15]:
df = fix_null(df)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,62879960.0,145.52,146.360,146.950,145.25,8.0,2021.0,64786620.0,147.36,145.81,...,2021.0,59773010.0,178.200,179.470,180.57,178.09,12.0,2021.0,False,False
1,64786620.0,147.36,145.810,148.045,145.18,8.0,2021.0,56368270.0,146.95,147.27,...,2021.0,64062260.0,177.570,178.085,179.23,177.26,12.0,2021.0,False,True
2,56368270.0,146.95,147.270,147.790,146.28,8.0,2021.0,46397670.0,147.06,146.98,...,2021.0,104701200.0,182.010,177.830,182.88,177.71,1.0,2022.0,False,True
3,46397670.0,147.06,146.980,147.840,146.17,8.0,2021.0,54126810.0,146.14,146.35,...,2022.0,99310440.0,179.700,182.630,182.94,179.12,1.0,2022.0,True,False
4,54126810.0,146.14,146.350,147.110,145.63,8.0,2021.0,48908690.0,146.09,146.20,...,2022.0,94537600.0,174.920,179.610,180.17,174.64,1.0,2022.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,64727190.0,142.91,147.075,147.300,141.92,12.0,2022.0,69721090.0,140.94,142.19,...,2023.0,45326870.0,171.770,173.050,173.54,171.60,5.0,2023.0,True,False
341,69721090.0,140.94,142.190,143.370,140.00,12.0,2022.0,62128340.0,142.65,142.36,...,2023.0,53724500.0,173.555,173.020,174.03,171.90,5.0,2023.0,False,False
342,62128340.0,142.65,142.360,143.520,141.10,12.0,2022.0,76097010.0,142.16,142.34,...,2023.0,49514680.0,173.750,173.850,174.59,172.17,5.0,2023.0,False,False
343,76097010.0,142.16,142.340,145.570,140.90,12.0,2022.0,70462650.0,144.49,142.70,...,2023.0,45533140.0,172.570,173.620,174.06,171.00,5.0,2023.0,False,True


In [16]:
target_percent_inc, target_best_day,target_worst_sell,target_day_high,target_day_low = get_targets(df,KF)
np.average(target_day_high)

156.86784057971013

In [17]:
np.average(target_day_low)

152.8421011594203

In [18]:
fix_normalize(apple2,df)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,-1.0,2.0,2.0,2.0,2.0,12.0,2021.0,False,False
1,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,-1.0,2.0,2.0,2.0,2.0,12.0,2021.0,False,True
2,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,1.0,2.0,2.0,2.0,2.0,1.0,2022.0,False,True
3,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2022.0,1.0,2.0,2.0,2.0,2.0,1.0,2022.0,True,False
4,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2022.0,0.0,2.0,2.0,2.0,2.0,1.0,2022.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,2.0,1.0,2.0,5.0,2023.0,True,False
341,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,2.0,1.0,1.0,2.0,5.0,2023.0,False,False
342,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,2.0,2.0,1.0,2.0,5.0,2023.0,False,False
343,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,2.0,1.0,1.0,5.0,2023.0,False,True


In [19]:
drop_future(df,KF)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,1.0,1.0,1.0,1.0,1.0,11.0,2021.0,False,False
1,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,1.0,1.0,1.0,0.0,1.0,11.0,2021.0,False,True
2,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,-0.0,1.0,1.0,0.0,1.0,11.0,2021.0,False,True
3,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,-0.0,0.0,0.0,0.0,0.0,11.0,2021.0,True,False
4,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,2021.0,-1.0,-1.0,-1.0,...,2021.0,0.0,0.0,0.0,0.0,0.0,11.0,2021.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,True,False
341,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,False,False
342,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,False,False
343,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,False,True


In [20]:
def round_low(y):
    for i in range(len(y)):
        y[i] = round(y[i])

In [21]:
#For better accuracy
def round_target(y,floor):
    for i in range(len(y)):
        curr = y[i]
        if 0<curr<floor:
            y[i] = 1
        elif -floor<curr<0:
            y[i] = -1
        else:
            y[i] = curr//floor
            if y[i] > 3:
                y[i] = 3
            if y[i] < -3:
                y[i] = -3

In [22]:
#finds min percent gains over KF days to double money in 1 year
def double_yearly(KF):
    TRADING_DAYS = 252
    periods = TRADING_DAYS/KF
    perc = (-1+2**(1/periods))*100
    return perc

In [23]:
def round_day(y):
    for i in range(len(y)):
        y[i] = y[i]//5

In [24]:
X = df
y = copy(target_percent_inc)
y_sell = copy(target_worst_sell)
y_day = copy(target_best_day)
round_day(y_day)
round_target(y,double_yearly(KF))
round_target(y_sell,double_yearly(KF))
y_sell

[1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 3.0,
 3.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1.0,
 1,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 0.0,
 -0.0,
 1,
 1,
 1,
 1,
 0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

In [26]:
X['WeekEnd']

0      False
1       True
2       True
3      False
4       True
       ...  
340    False
341    False
342    False
343     True
344    False
Name: WeekEnd, Length: 345, dtype: bool

In [27]:
clf_low = MLPClassifier()

In [28]:
clf = MLPClassifier(random_state=None)
clf_day = MLPClassifier(random_state=None)
clf_sell = MLPClassifier(random_state=1)

In [29]:
round_low(target_day_low)
target_day_low

[159,
 160,
 156,
 159,
 160,
 165,
 158,
 160,
 164,
 168,
 171,
 174,
 175,
 176,
 172,
 172,
 171,
 170,
 167,
 169,
 172,
 175,
 177,
 179,
 178,
 178,
 177,
 178,
 179,
 175,
 172,
 171,
 168,
 171,
 175,
 172,
 171,
 169,
 166,
 164,
 162,
 155,
 157,
 158,
 158,
 163,
 170,
 172,
 173,
 172,
 171,
 171,
 171,
 175,
 172,
 168,
 167,
 170,
 170,
 168,
 166,
 162,
 160,
 152,
 161,
 162,
 162,
 163,
 166,
 162,
 159,
 156,
 159,
 156,
 154,
 150,
 150,
 154,
 158,
 160,
 163,
 165,
 168,
 170,
 173,
 172,
 176,
 177,
 174,
 172,
 174,
 174,
 170,
 170,
 169,
 166,
 167,
 167,
 165,
 164,
 164,
 166,
 166,
 162,
 158,
 157,
 155,
 159,
 157,
 153,
 156,
 159,
 155,
 154,
 151,
 153,
 146,
 139,
 143,
 144,
 147,
 140,
 137,
 133,
 138,
 137,
 138,
 137,
 145,
 147,
 148,
 147,
 144,
 145,
 144,
 147,
 143,
 137,
 131,
 131,
 132,
 129,
 130,
 133,
 134,
 136,
 140,
 141,
 137,
 137,
 134,
 136,
 137,
 141,
 143,
 145,
 144,
 145,
 142,
 143,
 148,
 147,
 147,
 150,
 152,
 153,
 152

In [30]:
target_day_low[-1]

160

In [31]:
low_scores = cross_val_score(clf_low,X,target_day_low,cv=5,error_score='raise')

C:\Users\phill\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [32]:
low_scores

array([0.05797101, 0.05797101, 0.02898551, 0.02898551, 0.01449275])

In [33]:
scores = cross_val_score(clf, X, y, cv=5) #noticable improvement in variance and aveerage after adding week start and end
#except for the fact that without random_state=0 it can still be pretty bad...
#I know this is really bad practice, but for now I don't care
scores

array([0.56521739, 0.27536232, 0.75362319, 0.27536232, 0.55072464])

In [34]:
scores_sell = cross_val_score(clf_sell, X, y_sell, cv=5)
#The model is better at predicting the best case than the worst case for some reason
#(If I can buy a daily low and sell a daily high, the model gives me a more accurate prediction for percent increase)
#than if I buy a daily high and sell a daily low
scores_sell

array([0.8115942 , 0.8115942 , 0.8115942 , 0.8115942 , 0.79710145])

In [35]:
scores_day = cross_val_score(clf_day, X, y_day, cv=3)#not really any better
scores_day

array([0.31304348, 0.22608696, 0.11304348])

In [36]:
clf.fit(X,y)
clf_day.fit(X,y_day)
clf_sell.fit(X,y_sell)

MLPClassifier(random_state=1)

In [37]:
uptodate = pd.read_csv(path6+"apple.csv")
uptodate

,Date,Close/Last,Volume,Open,High,Low
0,05/19/2023,$175.16,55809480,$176.39,$176.39,$174.94
1,05/18/2023,$175.05,65496660,$173.00,$175.24,$172.58
2,05/17/2023,$172.69,57951600,$171.71,$172.925,$170.4201
3,05/16/2023,$172.07,42110290,$171.99,$173.1383,$171.7991
4,05/15/2023,$172.07,37266660,$173.16,$173.21,$171.47
...,...,...,...,...,...,...
245,05/27/2022,$149.64,90978500,$145.39,$149.68,$145.26
246,05/26/2022,$143.78,90601550,$137.39,$144.34,$137.14
247,05/25/2022,$140.52,92482700,$138.43,$141.785,$138.34
248,05/24/2022,$140.36,104132700,$140.805,$141.97,$137.33


In [38]:
u2 = uptodate.copy(deep=True)
fix_dollars(u2)
u2 = u2.loc[::-1].reset_index(drop=True)
u2

,Date,Volume,Close,Open,High,Low
0,05/23/2022,117726300,143.11,137.790,143.2600,137.6500
1,05/24/2022,104132700,140.36,140.805,141.9700,137.3300
2,05/25/2022,92482700,140.52,138.430,141.7850,138.3400
3,05/26/2022,90601550,143.78,137.390,144.3400,137.1400
4,05/27/2022,90978500,149.64,145.390,149.6800,145.2600
...,...,...,...,...,...,...
245,05/15/2023,37266660,172.07,173.160,173.2100,171.4700
246,05/16/2023,42110290,172.07,171.990,173.1383,171.7991
247,05/17/2023,57951600,172.69,171.710,172.9250,170.4201
248,05/18/2023,65496660,175.05,173.000,175.2400,172.5800


In [39]:
fix_date(u2)

In [40]:
u2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,117726300,143.11,137.790,143.2600,137.6500,5,2022,True,False
1,104132700,140.36,140.805,141.9700,137.3300,5,2022,False,False
2,92482700,140.52,138.430,141.7850,138.3400,5,2022,False,False
3,90601550,143.78,137.390,144.3400,137.1400,5,2022,False,False
4,90978500,149.64,145.390,149.6800,145.2600,5,2022,False,True
...,...,...,...,...,...,...,...,...,...
245,37266660,172.07,173.160,173.2100,171.4700,5,2023,True,False
246,42110290,172.07,171.990,173.1383,171.7991,5,2023,False,False
247,57951600,172.69,171.710,172.9250,170.4201,5,2023,False,False
248,65496660,175.05,173.000,175.2400,172.5800,5,2023,False,False


In [41]:
dfu = fix_shift(u2,KP,0,updated=True)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,117726300.0,143.11,137.790,143.2600,137.6500,5.0,2022.0,True,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,104132700.0,140.36,140.805,141.9700,137.3300,5.0,2022.0,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,92482700.0,140.52,138.430,141.7850,138.3400,5.0,2022.0,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,90601550.0,143.78,137.390,144.3400,137.1400,5.0,2022.0,False,False
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,90978500.0,149.64,145.390,149.6800,145.2600,5.0,2022.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,80223630.0,137.87,135.280,138.020,134.22,1.0,2023.0,81760310.0,141.11,138.120,...,2023.0,37266660.0,172.07,173.160,173.2100,171.4700,5.0,2023.0,True,False
246,81760310.0,141.11,138.120,143.315,137.90,1.0,2023.0,66435140.0,142.53,140.305,...,2023.0,42110290.0,172.07,171.990,173.1383,171.7991,5.0,2023.0,False,False
247,66435140.0,142.53,140.305,143.160,140.30,1.0,2023.0,65799350.0,141.86,140.890,...,2023.0,57951600.0,172.69,171.710,172.9250,170.4201,5.0,2023.0,False,False
248,65799350.0,141.86,140.890,142.430,138.81,1.0,2023.0,54105070.0,143.96,143.170,...,2023.0,65496660.0,175.05,173.000,175.2400,172.5800,5.0,2023.0,False,False


In [42]:
dfu.dropna(inplace=True)
dfu = dfu.reset_index(drop=True)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,117726300.0,143.11,137.790,143.260,137.65,5.0,2022.0,104132700.0,140.36,140.805,...,2022.0,90481110.0,152.37,154.65,155.2400,151.3800,9.0,2022.0,False,False
1,104132700.0,140.36,140.805,141.970,137.33,5.0,2022.0,92482700.0,140.52,138.430,...,2022.0,162278800.0,150.70,151.21,151.3500,148.3700,9.0,2022.0,False,True
2,92482700.0,140.52,138.430,141.785,138.34,5.0,2022.0,90601550.0,143.78,137.390,...,2022.0,81474250.0,154.48,149.31,154.5600,149.1000,9.0,2022.0,True,False
3,90601550.0,143.78,137.390,144.340,137.14,5.0,2022.0,90978500.0,149.64,145.390,...,2022.0,107689800.0,156.90,153.40,158.0800,153.0800,9.0,2022.0,False,False
4,90978500.0,149.64,145.390,149.680,145.26,5.0,2022.0,103718400.0,148.84,149.070,...,2022.0,101696800.0,153.72,157.34,158.7400,153.6000,9.0,2022.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,80223630.0,137.87,135.280,138.020,134.22,1.0,2023.0,81760310.0,141.11,138.120,...,2023.0,37266660.0,172.07,173.16,173.2100,171.4700,5.0,2023.0,True,False
167,81760310.0,141.11,138.120,143.315,137.90,1.0,2023.0,66435140.0,142.53,140.305,...,2023.0,42110290.0,172.07,171.99,173.1383,171.7991,5.0,2023.0,False,False
168,66435140.0,142.53,140.305,143.160,140.30,1.0,2023.0,65799350.0,141.86,140.890,...,2023.0,57951600.0,172.69,171.71,172.9250,170.4201,5.0,2023.0,False,False
169,65799350.0,141.86,140.890,142.430,138.81,1.0,2023.0,54105070.0,143.96,143.170,...,2023.0,65496660.0,175.05,173.00,175.2400,172.5800,5.0,2023.0,False,False


In [43]:
fix_normalize(apple2,dfu)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,1.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,1.0,-1.0,-1.0,...,2022.0,0.0,-0.0,0.0,-0.0,-0.0,9.0,2022.0,False,False
1,1.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,0.0,-1.0,-1.0,...,2022.0,3.0,-0.0,-0.0,-0.0,-0.0,9.0,2022.0,False,True
2,0.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,0.0,-1.0,-1.0,...,2022.0,-0.0,-0.0,-0.0,-0.0,-0.0,9.0,2022.0,True,False
3,0.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,0.0,-0.0,-1.0,...,2022.0,1.0,0.0,-0.0,0.0,0.0,9.0,2022.0,False,False
4,0.0,-0.0,-1.0,-1.0,-1.0,5.0,2022.0,1.0,-0.0,-0.0,...,2022.0,1.0,-0.0,0.0,0.0,0.0,9.0,2022.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,-0.0,-1.0,-2.0,-1.0,-1.0,1.0,2023.0,-0.0,-1.0,-1.0,...,2023.0,-2.0,1.0,2.0,1.0,2.0,5.0,2023.0,True,False
167,-0.0,-1.0,-1.0,-1.0,-1.0,1.0,2023.0,-1.0,-1.0,-1.0,...,2023.0,-2.0,1.0,1.0,1.0,2.0,5.0,2023.0,False,False
168,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2023.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,5.0,2023.0,False,False
169,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2023.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,2.0,1.0,2.0,2.0,5.0,2023.0,False,False


In [44]:
point = np.array(dfu.iloc[-1]).reshape(1,-1)
point

array([[-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 2023.0, -0.0, -1.0, -1.0,
        -1.0, -1.0, 1.0, 2023.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0,
        2023.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 2023.0, -0.0, -1.0,
        -1.0, -1.0, -1.0, 2.0, 2023.0, 1.0, -0.0, -0.0, -0.0, -0.0, 2.0,
        2023.0, 3.0, 0.0, -1.0, 0.0, -0.0, 2.0, 2023.0, -0.0, -0.0, -0.0,
        -0.0, -0.0, 2.0, 2023.0, 0.0, 0.0, -0.0, -0.0, -0.0, 2.0, 2023.0,
        -1.0, -0.0, -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0, -0.0,
        -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0, -0.0, -0.0, -0.0, 2.0,
        2023.0, -1.0, -0.0, -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0,
        -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, 0.0, -0.0, -0.0, 0.0, 2.0,
        2023.0, -1.0, -0.0, -0.0, 0.0, 0.0, 2.0, 2023.0, -1.0, -0.0,
        -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0, -0.0, -0.0, -0.0, 2.0,
        2023.0, -1.0, -0.0, -0.0, -1.0, -0.0, 2.0, 2023.0, -1.0, -0.0,
        -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -1.0, -1.0, -1.0, -1.0, 2.0,
   

In [45]:
clf.predict(point)#Now it's saying apple will go down.
#With random state = 0 and 3 cross val scores approx 70%, predict apple will go up by less than ~8%
#With random state = 1 and 2 good scores 1 bad, predict apple will go down by less than 8%
# (given tomorrow's high, the highest low of the next 25 days will be down by less than 8%)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([-1.])

In [46]:
clf_sell.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([1.])

In [47]:
clf_day.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)